# [Collision Detection](https://www.youtube.com/watch?v=1lHyIFgCSAA)

In [1]:
import pygame  # Import the pygame library

# Initialize Pygame
pygame.init()

# Set up the screen size
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# Set the title of the window
pygame.display.set_caption("Move the Red Square")

# Colors (RGB format)
BLACK = (0, 0, 0)       # Background color
RED = (255, 0, 0)       # Red square color
YELLOW = (255, 255, 0)  # Yellow square color

# Square size (both red and yellow will be the same size)
SQUARE_SIZE = 50

# Start position for the red square (upper left corner area)
red_x = 100
red_y = 100

# Position the yellow square in the center of the screen
yellow_x = (SCREEN_WIDTH - SQUARE_SIZE) // 2
yellow_y = (SCREEN_HEIGHT - SQUARE_SIZE) // 2

# Speed for moving the red square
SQUARE_SPEED = 5

# Function to check if two squares overlap (touching each other)
def squares_overlap(x1, y1, x2, y2, size):
    if (x1 < x2 + size and          # Red left edge is to the left of yellow right edge
        x1 + size > x2 and          # Red right edge is to the right of yellow left edge
        y1 < y2 + size and          # Red top edge is above yellow bottom edge
        y1 + size > y2):            # Red bottom edge is below yellow top edge
        return True
    else:
        return False

# Main game loop
running = True
clock = pygame.time.Clock()  # Controls how fast the game runs

while running:
    # Check for quit event (close window)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Get keys being pressed
    keys = pygame.key.get_pressed()

    # Store the red square's new position based on keys
    new_red_x = red_x
    new_red_y = red_y

    if keys[pygame.K_LEFT]:
        new_red_x -= SQUARE_SPEED
    if keys[pygame.K_RIGHT]:
        new_red_x += SQUARE_SPEED
    if keys[pygame.K_UP]:
        new_red_y -= SQUARE_SPEED
    if keys[pygame.K_DOWN]:
        new_red_y += SQUARE_SPEED

    # Keep the red square inside the screen (don't go off the edges)
    if new_red_x < 0:
        new_red_x = 0
    if new_red_x > SCREEN_WIDTH - SQUARE_SIZE:
        new_red_x = SCREEN_WIDTH - SQUARE_SIZE
    if new_red_y < 0:
        new_red_y = 0
    if new_red_y > SCREEN_HEIGHT - SQUARE_SIZE:
        new_red_y = SCREEN_HEIGHT - SQUARE_SIZE

    # Only update red square's position if it does NOT overlap the yellow square
    if not squares_overlap(new_red_x, new_red_y, yellow_x, yellow_y, SQUARE_SIZE):
        red_x = new_red_x
        red_y = new_red_y

    # Draw everything
    screen.fill(BLACK)  # Clear screen
    pygame.draw.rect(screen, RED, (red_x, red_y, SQUARE_SIZE, SQUARE_SIZE))  # Draw red square
    pygame.draw.rect(screen, YELLOW, (yellow_x, yellow_y, SQUARE_SIZE, SQUARE_SIZE))  # Draw yellow square

    # Update the display
    pygame.display.flip()

    # Control the speed of the loop (frames per second)
    clock.tick(60)

# Quit Pygame when the loop ends
pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# https://chatgpt.com/share/67ec3ca8-08e4-8011-a4e2-04d49b8b5808
# https://www.youtube.com/watch?v=D2M8jTtKi44

import pygame
import random
import math

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
BALL_RADIUS = 10
BALL_COUNT = 100
GRID_SIZE = BALL_RADIUS * 2  # Cell size slightly larger than ball diameter

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Pygame screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

class Ball:
    def __init__(self, x, y, vx, vy, color):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.color = color

    def move(self):
        """Move the ball and bounce off walls."""
        self.x += self.vx
        self.y += self.vy

        # Bounce off walls
        if self.x - BALL_RADIUS < 0 or self.x + BALL_RADIUS > WIDTH:
            self.vx = -self.vx
        if self.y - BALL_RADIUS < 0 or self.y + BALL_RADIUS > HEIGHT:
            self.vy = -self.vy

    def draw(self):
        """Draw the ball on the screen."""
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), BALL_RADIUS)

def detect_collisions(balls):
    """Detect and resolve ball collisions using spatial hashing."""
    grid = {}  # Dictionary for grid-based collision detection

    # Assign each ball to grid cells
    for ball in balls:
        cell_x = int(ball.x // GRID_SIZE)
        cell_y = int(ball.y // GRID_SIZE)
        cell_key = (cell_x, cell_y)

        if cell_key not in grid:
            grid[cell_key] = []
        grid[cell_key].append(ball)

    # Check for collisions within the same or adjacent grid cells
    for cell_key, cell_balls in grid.items():
        neighbor_cells = [
            (cell_key[0] + dx, cell_key[1] + dy)
            for dx in [-1, 0, 1] for dy in [-1, 0, 1]
        ]  # Check surrounding cells

        nearby_balls = []
        for neighbor in neighbor_cells:
            if neighbor in grid:
                nearby_balls.extend(grid[neighbor])

        # Check for collisions within the nearby balls
        for i, ball1 in enumerate(cell_balls):
            for ball2 in nearby_balls[i + 1:]:  # Avoid duplicate checks
                dx = ball1.x - ball2.x
                dy = ball1.y - ball2.y
                distance_sq = dx * dx + dy * dy
                if distance_sq < (2 * BALL_RADIUS) ** 2:  # Collision detected
                    resolve_collision(ball1, ball2)

def resolve_collision(ball1, ball2):
    """Resolve collision between two balls using elastic collision physics."""
    dx = ball2.x - ball1.x
    dy = ball2.y - ball1.y
    distance = math.sqrt(dx**2 + dy**2)

    if distance == 0:
        return  # Avoid division by zero

    # Normalized collision vector
    nx, ny = dx / distance, dy / distance

    # Relative velocity
    vx_rel = ball2.vx - ball1.vx
    vy_rel = ball2.vy - ball1.vy

    # Dot product to check if they are moving towards each other
    dot_product = vx_rel * nx + vy_rel * ny
    if dot_product > 0:
        return  # Balls are moving away, no need to resolve

    # Swap velocities in the normal direction
    ball1.vx += dot_product * nx
    ball1.vy += dot_product * ny
    ball2.vx -= dot_product * nx
    ball2.vy -= dot_product * ny

# Create balls
balls = [
    Ball(
        random.randint(BALL_RADIUS, WIDTH - BALL_RADIUS),
        random.randint(BALL_RADIUS, HEIGHT - BALL_RADIUS),
        random.uniform(-3, 3),
        random.uniform(-3, 3),
        RED if i % 2 == 0 else BLUE
    )
    for i in range(BALL_COUNT)
]

# Game loop
running = True
while running:
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Move and detect collisions
    for ball in balls:
        ball.move()
    detect_collisions(balls)

    # Draw balls
    for ball in balls:
        ball.draw()

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
